# Enrichment analysys
### Over representation analysis
* Simplified ilustrative example
* Followed by real case
### GSEA
* Practical using python package

# Supervised machine learning
### Linear regression
### Logistic regression
### SVM
### Overfitting and Bias vs Variance -> Cross validation
### Dimentionality of the input -> PCA

# Unsupervised machine learning
### K-means clustering
* Using TCGA data, HR+/-

# Pathway analysis

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import gseapy as gp
from gseapy.plot import gseaplot
import qvalue

from ipywidgets import interact, interact_manual
from ipywidgets import IntSlider, FloatSlider, Dropdown, Text

interact_enrich=interact_manual.options(manual_name="Enrichment analysis")
interact_plot=interact_manual.options(manual_name="Plot")
interact_calc=interact_manual.options(manual_name="Calculate tests")

In [ ]:
clinical_data = pd.read_csv('data/brca_clin.tsv.gz', sep ='\t', index_col=2)
clinical_data = clinical_data.iloc[4:,1:]
expression_data = pd.read_csv('data/brca.tsv.gz', sep ='\t', index_col=1)
expression_data = expression_data.iloc[:,2:].T

In [ ]:
def differential_test(clinical_df, expression_df, separator, cond1, cond2):
    p_vals = pd.DataFrame(columns = ['p'])
    for gene in expression_df.columns:
        try:
            expression = expression_df[gene]
        except:
            print('Gene not found in data')
        try:
            group1 = clinical_df[separator] == cond1
            index1 = clinical_df[group1].index
            group2 = clinical_df[separator] == cond2
            index2 = clinical_df[group2].index
        except:
            print('Clinical condition wrong')
        expression1 = expression[index1].dropna()
        expression2 = expression[index2].dropna()
        p_val = sp.stats.ttest_ind(expression1, expression2).pvalue
        if p_val == p_val:
            p_vals.loc[gene,'p'] = p_val
    return p_vals

def plot_hist(stats, bins):
    stats = np.array(stats)
    plt.hist(stats, bins = bins)
    plt.show()


def interact_multiple_gene_ttest(Criteria, Group_1, Group_2):
    global BRCA_tests
    BRCA_tests = differential_test(clinical_data, expression_data, Criteria, Group_1, Group_2)
    BRCA_tests = qvalue.qvalues(BRCA_tests)
    plot_hist(BRCA_tests['p'].values, 20)
    with pd.option_context('display.max_rows', None):
        display(BRCA_tests)
        
def ORA(tests, threshold, pathway_db=['KEGG_2019_Human'], stat = 'p'):
    background=set(tests.index)
    gene_list = list(tests.loc[tests[stat]<threshold,stat].index)

    output_enrichr=pd.DataFrame()
    enr=gp.enrichr(
                    gene_list=gene_list,
                    gene_sets=pathway_db,
                    background=background,
                    outdir = None
                )
    results = enr.results[["P-value","Overlap","Term"]].rename(columns={"P-value": "p"})
    return qvalue.qvalues(results)

pathway_db_choice = gp.get_library_name()

        
def interact_ORA(Threshold, Pathway_DB, Statistic):
    threshold = float(threshold)
    results = ORA(BRCA_tests, Threshold, Pathway_DataBase, stat = Statistic)
    with pd.option_context('display.max_rows', None):
        display(results)

In [ ]:
interact_calc(interact_multiple_gene_ttest, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_enrich(interact_ORA, Threshold = '5e-2' , Pathway_DB = pathway_db_choice, Statistic=['p','q'])

In [ ]:
def gsea(tests, pathway_db = 'KEGG_2019_Human' ):
    pre_res = gp.prerank(rnk=tests['p'], 
                    gene_sets=pathway_db,
                    processes=4,
                    permutation_num=100, # reduce number to speed up testing
                    outdir=None, format='png')
    return pre_res

def interact_gsea(Pathway_DB):
    global Results_gsea
    Results_gsea = gsea(BRCA_tests, pathway_db=Pathway_DB)
    with pd.option_context('display.max_rows', None):
        display(Results_gsea.res2d[['pval', 'fdr']])

In [ ]:
interact_calc(interact_multiple_gene_ttest, Criteria=Text('Surgical procedure first'), Group_1 = Text('Simple Mastectomy'), Group_2=Text('Lumpectomy'))
interact_enrich(interact_gsea, Pathway_DB = pathway_db_choice)

In [ ]:
terms = Results_gsea.res2d.index[0]

gseaplot(rank_metric=Results_gsea.ranking, term=terms, **Results_gsea.results[terms])

In [ ]:
Results_gsea.results['Primary bile acid biosynthesis_Homo sapiens_hsa00120']
Results_gsea.ranking

# Machine learning, SVMs

# References

Weinstein, John N., et al. 'The cancer genome atlas pan-cancer analysis project.' Nature genetics 45.10 (2013): 1113-1120.

Patrício, Miguel, et al. “Using Resistin, Glucose, Age and BMI to Predict the Presence of Breast Cancer.” BMC Cancer, vol. 18, no. 1, Jan. 2018, p. 29. BioMed Central, doi:10.1186/s12885-017-3877-1.